In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    access_token = userdata.get('DEFORMER_TOKEN')
    !pip install git+https://$access_token@github.com/ay94/deformer-extractor.git@main

  Cloning https://****@github.com/ay94/deformer-extractor.git (to revision main) to /tmp/pip-req-build-l3l4x8si
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/ay94/deformer-extractor.git' /tmp/pip-req-build-l3l4x8si
  Resolved https://****@github.com/ay94/deformer-extractor.git to commit a44060b0cb952793aefcaf95b2ff6aad5e760bc2
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 k

In [22]:
from experiment_utils import env_setup
from experiment_utils.utils import  FileHandler
from experiment_utils.pipelines import  ExperimentInitializer

In [20]:
base_folder = env_setup.init(drive_folder='My Drive', drive_mount='ahmed')

Mounted at /content/ahmed


In [23]:
experiment_base_folder = base_folder / 'Final Year Experiments/Thesis-Experiments/Experiments/test_merge'
experiments_fh = FileHandler(experiment_base_folder.parent)
experiments = experiments_fh.load_yaml('experiments.yaml')

In [28]:
variant = "ANERCorp_CamelLab_arabertv02"
variant_data = experiments.get(variant)
# # dataset_name = "conll2003"
# # model_name = "bert"
# # model_path = "bert-base-cased"
# # variant = "conll2003_bert"

# dataset_name = "ANERCorp_CamelLab"
# model_name = "arabertv02"
# model_path = "aubmindlab/bert-base-arabertv02"
# variant = "ANERCorp_CamelLab_arabertv02"

experiment_config = {
    "experiment_name": "DashboardTest",
    "corpora_path": "ExperimentData",
    "variant": variant,
    "dataset_name": variant_data['dataset_name'],
    "model_name": variant_data['model_name'],
    "model_path": variant_data['model_path'],
    "results_config": "results_config.yaml",
    "extraction_config": "extraction_config.yaml",
    "fine_tuning_config": "fine_tuning_config.yaml"
}


extraction_config = {
    "fine_tuning": {
        "args": {
            "train_batch_size": 16,
            "test_batch_size": 8,
            "shuffle": True,
            "num_workers": 2,
            "epochs": 4,
            "splits": 4,
            "learning_rate": 5e-5,
            "warmup_ratio": 0.1,
            "max_grad_norm": 1.0,
            "accumulation_steps": 1,
            "logging_step": 50
        },
        "model": {
            "model_path": variant_data['model_path'],
            "dropout_rate": 0.1,
            "enable_attentions": False,
            "enable_hidden_states": False,
            "initialize_output_layer": False
        },
        "evaluation": {
            "scheme": None,
            "mode": None
        }
    },
    "extraction": {
        "tokenization": {
            "tokenizer_path": variant_data['model_path'],
            "preprocessor_path": None,
            "max_seq_len": 256,
            "strategy": {
                "type": "core",
                "index": 0,
                "schema": None
            }
        },
        "umap": {
            "n_neighbors": 15,
            "min_dist": 0.1,
            "metric": "cosine",
            "random_state": 1,
            "verbose": True,
            "normalize_embeddings": False
        },
        "clustering": {
            "init_method": "k-means++",
            "n_init": 10,
            "random_state": 1,
            "n_clusters": [3, 4, 9],
            "n_clusters_map": {
                3: "boundary_clusters",
                4: "entity_clusters",
                9: "token_clusters"
            },
            "silhouette_metric": "cosine",
            "norm": "l2"
        }
    }
}

results_config = {
    "results_dir": "extractions",
    "analysis_data": {
        "folder": "analysis",
        "filename": "analysis_data",
        "format": "json"
    },
    "entity_report": {
        "folder": "results",
        "filename": "entity_report",
        "format": "json"
    },
    "token_report": {
        "folder": "results",
        "filename": "token_report",
        "format": "json"
    },
    "results": {
        "folder": "results",
        "filename": "results",
        "format": "json"
    },
    "kmeans_results": {
        "folder": "results",
        "filename": "kmeans_results",
        "format": "json"
    },
    "entity_confusion_data": {
        "folder": "results",
        "filename": "entity_confusion_data",
        "format": "json"
    },
    "attention_weights_similarity": {
        "folder": "matrices",
        "filename": "attention_weights_similarity",
        "format": "json"
    },
    "centroids_avg_similarity_matrix": {
        "folder": "matrices",
        "filename": "centroids_avg_similarity_matrix",
        "format": "json"
    },
    "attention_similarity_matrix": {
        "folder": "matrices",
        "filename": "attention_similarity_matrix",
        "format": "json"
    },
    "train_df": {
        "folder": "analysis",
        "filename": "train_df",
        "format": "json"
    },
}

fine_tuning_config = {
    "save_dir": "fine_tuning",  # Base directory for saving outputs
    "metrics": {
        "folder": "metrics",
        "filename": "evaluation_metrics",
        "format": "json"
    },
    "model": {
        "state_dict": {
            "filename": "model_state_dict",
            "format": "pth"
        },
        "binary": {
            "filename": "model_binary",
            "format": "bin"
        }
    },

}

In [29]:
# setup experiment folder
manager = ExperimentInitializer(experiment_base_folder, experiment_config, extraction_config, results_config, fine_tuning_config)
manager.setup_experiment()

In [30]:
drive.flush_and_unmount()